# 01 Example - Property Scans with Point Data

In [1]:
import pyscan as ps
import numpy as np
import matplotlib.pyplot as plt

## Setup devices

In [2]:
devices = ps.ItemAttribute()

devices.v1 = ps.TestVoltage() # Device 1

## Define a measure function

In [3]:
def get_voltage_data(expt):
    """
    Reads the voltage from v1, v2, and v3 devices. Also adds a calculated value vsum.
    """

    runinfo = expt.runinfo
    devices = expt.devices

    d = ps.ItemAttribute()

    d.v1_readout = devices.v1.voltage

    d.vres = (d.v1_readout - 4)**2
    
    return d

# Define an optimization function

In [4]:
def voltage_result_grad(v):
    return 2 * v - 8

In [5]:
def gradient_descent(f_val, f_grad=None, learning_rate=1.):
    return f_val - learning_rate * f_grad(f_val)

# Experiments

In [6]:
runinfo =ps.RunInfo()
runinfo.scan0 = ps.OptimizeScan({'v1': 1.}, prop='voltage', f_optimizer=gradient_descent,
                                optimizer_inputs=['vres'],
                                optimizer_hyperparameters={'f_grad': voltage_result_grad, 'learning_rate': .1},
                                iteration_max=100)

runinfo.measure_function = get_voltage_data

expt = ps.Experiment(runinfo, devices, time=True)
expt.start_thread()

Exception in thread Thread-3 (run):
Traceback (most recent call last):
  File "c:\Users\djbasil\AppData\Local\miniconda3\envs\ps\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\djbasil\AppData\Local\miniconda3\envs\ps\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\djbasil\AppData\Local\miniconda3\envs\ps\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\djbasil\Documents\GitHub\pyscan\pyscan\measurement\experiment.py", line 273, in run
    self.generic_experiment()
    ~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\djbasil\Documents\GitHub\pyscan\pyscan\measurement\experiment.py", line 42, in generic_experiment
    self.runinfo['t{}'.format(i)] = np.zeros(self.runinfo.dims)
                                             ^^^^^^^^^^^^^^^^^
  File "C:\Us

In [7]:
ps.live_plot1D(expt, data_name="v1_readout")

KeyboardInterrupt: 